In [ ]:
import zipfile
from urllib import request
import pathlib
import collections
import warnings

import matplotlib.pyplot as plt
import imageio

import ipywidgets

In [ ]:
url = 'https://github.com/pymedphys/data/releases/download/mini-lung/mini-lung-medical-decathlon.zip'
filename = url.split('/')[-1]

In [ ]:
request.urlretrieve(url, filename)

In [ ]:
data_path = pathlib.Path('data')

In [ ]:
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall(data_path)

In [ ]:
image_paths = sorted(data_path.glob('**/*_image.png'))

mask_paths = [
    path.parent.joinpath(path.name.replace('_image.png', '_mask.png'))
    for path in image_paths
]

In [ ]:
image_mask_pairs = collections.defaultdict(lambda: [])

for image_path, mask_path in zip(image_paths, mask_paths):
    patient_label = image_path.parent.name
    
    image = imageio.imread(image_path)
    mask = imageio.imread(mask_path)
    
    image_mask_pairs[patient_label].append((image, mask))

In [ ]:
def get_contours_from_mask(mask, contour_level=128):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning)
        fig, ax = plt.subplots()
        cs = ax.contour(range(mask.shape[0]), range(mask.shape[0]), mask, [contour_level])

    contours = [path.vertices for path in cs.collections[0].get_paths()]
    plt.close(fig)

    return contours

In [ ]:
def display(patient_label, chosen_slice):
    image = image_mask_pairs[patient_label][chosen_slice][0]
    mask = image_mask_pairs[patient_label][chosen_slice][1]

    plt.figure(figsize=(10,10))
    plt.imshow(image)

    contours = get_contours_from_mask(mask)
    for contour in contours:
        plt.plot(*contour.T, 'k')

In [ ]:
def view_patient(patient_label):
    def view_slice(chosen_slice):
        display(patient_label, chosen_slice)
        
    number_of_slices = len(image_mask_pairs[patient_label])
    
    ipywidgets.interact(view_slice, chosen_slice=ipywidgets.IntSlider(min=0, max=number_of_slices, step=1, value=0));

In [ ]:
ipywidgets.interact(view_patient, patient_label=image_mask_pairs.keys());

In [ ]:
# Issue with how matplotlib interprets small masks
# Need to get to the bottom of this, or use a different tool for mask --> contour
display('lung_003', 138)